Rebuilding Steps, Example Network, Integrated in SNMan 'Multi' Rebuild
====================
xxx

In [1]:
import warnings
warnings.filterwarnings('ignore')

import copy
import os
import sys
import networkx as nx
import numpy as np
import shutil

import snman
from snman.constants import *
from snman import osmnx_customized as oxc
from snman import *

#PERIMETER = '_paper2_example'
PERIMETER = '_debug'
#PERIMETER = 'zrh_north-west'
#PERIMETER = 'ebc_zrh_v01_10km'

data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', '_debug')

CRS_internal = 2056      # for Zurich

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [2]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)

street_graph.organize_edge_directions(G)

Load street graph


In [3]:
print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    #os.path.join(process_path, 'rebuilding_regions.gpkg'),
    os.path.join(inputs_path, 'rebuilding_regions', 'rebuilding_regions.gpkg'),
    only_active=True,
    #filter_ids=['zrh-4'],
    #filter_ids=['zrh-19'],
    #filter_ids=['zrh-main'],
    #filter_ids=['playground-1'],
    #filter_ids=['paper2_example'],
    crs=CRS_internal
)

print('Load access needs')
access_needs = snman.io.load_access_needs(
    os.path.join(inputs_path, 'switzerland', 'zurich', 'statpop', 'statpop17_zurich_city_agg_on_buildings.gpkg'),
    CRS_internal
)

access_needs['parking_spots_needed'] = access_needs['residents'] / 60

Load rebuilding regions
Load access needs


In [4]:
import geopandas as gpd
from snman import *
from snman.rebuilding import *

HLAs = multi_rebuild_regions(
    G, rebuilding_regions_gdf, access_needs,
    save_steps_path=os.path.join(export_path, 'steps'),
    save_steps_scaling_factor=2,
    gravity_iterations=10,
)
HLAs = list(filter(lambda HLA: HLA is not None, HLAs))

truncating street graph
done truncating
GIVEN LANES
REMOVE PARKING
iteration 1
REMOVE CAR LANES
iteration 1
removed (14035, 75178, '14035-75178-0-0')
iteration 2
fixed [4147, 75108, '4147-75108-1-0-1']
iteration 3
removed (11575, 4147, '4147-11575-0-0')
iteration 4
fixed [11575, 3509, '3509-11575-0-0-2']
iteration 5
fixed [16180, 4147, '4147-16180-0-0-2']
iteration 6
fixed [75108, 4147, '4147-75108-1-0-2']
iteration 7
removed (29877, 29887, '29877-29887-0-0')
iteration 8
removed (31803, 28732, '28732-31803-0-0')
iteration 9
removed (276, 66, '66-276-0-1')
iteration 10
removed (101, 67, '67-101-0-0')
iteration 11
fixed [101, 697, '101-697-0-2-1']
iteration 12
removed (75380, 71, '71-75380-0-0')
iteration 13
fixed [75380, 6310, '6310-75380-0-0-2']
iteration 14
fixed [16596, 16456, '16456-16596-0-0-2']
iteration 15
removed (1508, 76, '76-1508-0-0')
iteration 16
removed (75317, 72, '72-75317-0-0')
iteration 17
fixed [75317, 6310, '6310-75317-0-0-2']
iteration 18
fixed [75381, 90, '90-75381

In [12]:
import networkx as nx

Hs,Ls,As = zip(*HLAs)
A = nx.compose_all(As)
L = nx.compose_all(Ls)

snman.io.export_HLA(os.path.join(export_path, 'steps'), '_debug', L=L, A=A)

In [6]:
print('Reorder lanes')
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

print('Normalize cycling infrastructure')
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION)
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION_AFTER)

snman.street_graph.organize_edge_directions(G)

In [7]:
print('Add elevation')
G = oxc.elevation.add_node_elevations_raster(
    G,
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'ch_dhm_25', 'ch_dhm_2056.tif'),
    raster_crs=2056,
    #os.path.join(inputs_path, 'copernicus_dem', 'Copernicus_DSM_10_N41_00_W088_00', 'DEM', 'Copernicus_DSM_10_N41_00_W088_00_DEM.tif'),
    #raster_crs=4326, #Chicago
    graph_crs=CRS_internal,
    cpus=1
)
G = oxc.elevation.add_edge_grades(G, add_absolute=False)

In [8]:
if 0:

    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION)
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width', lanetypes={LANETYPE_CYCLING_LANE})
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width')
    snman.street_graph.lane_graph_to_street_graph(
        G, L,
        KEY_LANES_DESCRIPTION
    )
    
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width', lanetypes={LANETYPE_CYCLING_LANE})
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width')
    snman.street_graph.lane_graph_to_street_graph(
        G, L,
        KEY_LANES_DESCRIPTION_AFTER
    )

In [9]:
snman.space_allocation.generate_lane_stats(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.generate_lane_stats(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

In [10]:
if 1:
    print('export G')
    snman.io.export_street_graph(
        G,
        os.path.join(export_path, 'G_edges.gpkg'),
        os.path.join(export_path, 'G_nodes.gpkg')
    )

if 1:
    print('export L before')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_nodes.gpkg'),
        scaling=2
    )

if 1:
    print('export L after')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_rebuilt_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_rebuilt_nodes.gpkg'),
        scaling=2
    )

In [11]:
if 0:
    print('Load street graph')
    G = snman.io.load_street_graph(
        os.path.join(export_path, 'G_edges.gpkg'),
        os.path.join(export_path, 'G_nodes.gpkg'),
        crs=CRS_internal
    )
    
    street_graph.organize_edge_directions(G)